In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-cmicgthn
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-cmicgthn
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20045 sha256=888336eee38f5667cf97f118b68b4b6435523924e152abda8d415f3825d880f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhgaz10m/wheels/0b/96/56/27b17c903efc647c51e4f364bfc20aa67f8d3dccad63c4fb4e
Successfully built image-classifiers


In [47]:
#!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0-py2.py3-none-any.whl
     |████████████████████████████████| 31.2 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 121.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [13]:
import data_handler
import tensorflow as tf
import numpy as np
import pickle
import json
import os
import itertools

In [2]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/data'

tfrecord_files = np.asarray(data_handler.create_full_tfrecords_paths(data_dir))

# get train, val, test fold
with open(data_dir + '/sorted_dhs_incountry_folds.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

# get band stats
with open(data_dir + '/band_stats.json') as band_stats_file:
    band_stats = json.load(band_stats_file)
    
# tf.keras.backend.set_floatx('float16')

In [ ]:
def get_batch_stats(x):
    masks = x['outputs_mask']
    imgs = tf.cast(x['model_input'], dtype=tf.float32)
    print(imgs.shape)
    print(masks)
    print(7/0)
    #band_values = tf.reshape(imgs, [None, 10, 224*224, 8])
    frame_is = tf.argmax(masks, axis=1)
    
    slices = tf.stack([imgs[i, frame_i, ...] for i, frame_i in enumerate(frame_is)])
    
    batch_stats = {
        's1' : {},
        's2': {},
        'n': {}
    }
    
    slices = tf.cast(slices, dtype=tf.float32)
    slices2 = tf.math.square(slices)
    s1 = tf.reduce_sum(slices, axis=(0,1))
    s2 = tf.reduce_sum(slices2, axis=(0,1))
    
    
    bands = ['BLUE', 'GREEN', 'RED', 'SWIR1', 'SWIR2', 'TEMP1', 'NIR']
    for i, band in enumerate(bands):
        batch_stats['s1'][band] = s1[i]
        batch_stats['s2'][band] = s2[i]
        batch_stats['n'][band] = tf.constant(imgs.shape[0] * 224*224*8, dtype=tf.int32)
        
    is_viirs = frame_is > 7
    
    batch_stats['s1']['DMSP'] = tf.reduce_sum(slices[~is_viirs], axis=(0,1))[7]
    batch_stats['s2']['DMSP'] = tf.reduce_sum(slices2[~is_viirs], axis=(0,1))[7]
    batch_stats['n']['DMSP'] = tf.reduce_sum(tf.cast(~is_viirs, tf.int32))
    
    batch_stats['s1']['VIIRS'] = tf.reduce_sum(slices[is_viirs], axis=(0,1))[7]
    batch_stats['s2']['VIIRS'] = tf.reduce_sum(slices2[is_viirs], axis=(0,1))[7]
    batch_stats['n']['VIIRS'] = tf.reduce_sum(tf.cast(is_viirs, tf.int32))
    
    return batch_stats

In [222]:
%%time

def get_batch_stats(x):
    mask = x['outputs_mask']
    img = tf.cast(x['model_input'], dtype=tf.float32)
    #print(img.shape)
    #print(mask.shape)
    i = tf.argmax(mask)
    slice_i = img[i]
    #band_values = tf.reshape(imgs, [None, 10, 224*224, 8])
    #frame_is = tf.argmax(masks, axis=1)
    
    #slices = tf.stack([imgs[i, frame_i, ...] for i, frame_i in enumerate(frame_is)])
    
    batch_stats = {
        's1' : {
            'DMSP': 0,
            'VIIRS': 0
        },
        's2': {
            'DMSP': 0,
            'VIIRS': 0
        },
        'n': {
            'DMSP': 0,
            'VIIRS': 0
        }
    }
    
    # slice_i.shape == (224, 224, 8)
    s1 = tf.reduce_sum(slice_i, axis=(0,1))
    s2 = tf.reduce_sum(tf.math.square(slice_i), axis=(0,1))
    is_dmsp = i < 8
    return {'s1': s1, 's2': s2, 'is_dmsp': is_dmsp}
    

fold_files = tfrecord_files[fold_indices]
ds = get_dataset(fold_files, batch_size=0, n_of_frames=10, shuffle=False, normalize=False, max_epochs=0)
ds = ds.map(lambda x, _: get_batch_stats(x))

stats = {
    's1' : {
        'BLUE': 0, 
        'GREEN': 0, 
        'RED': 0, 
        'SWIR1': 0, 
        'SWIR2': 0, 
        'TEMP1': 0, 
        'NIR': 0,
        'DMSP': 0,
        'VIIRS': 0
    },
    's2': {
        'BLUE': 0, 
        'GREEN': 0, 
        'RED': 0, 
        'SWIR1': 0, 
        'SWIR2': 0, 
        'TEMP1': 0, 
        'NIR': 0,
        'DMSP': 0,
        'VIIRS': 0
    },
    'n': {
        'BLUE': 0, 
        'GREEN': 0, 
        'RED': 0, 
        'SWIR1': 0, 
        'SWIR2': 0, 
        'TEMP1': 0, 
        'NIR': 0,
        'DMSP': 0,
        'VIIRS': 0
    }
}
for x in ds:
    for i, band in enumerate(['BLUE', 'GREEN', 'RED', 'SWIR1', 'SWIR2', 'TEMP1', 'NIR']):   
        stats['s1'][band] += x['s1'][i].numpy()  
        stats['s2'][band] += x['s2'][i].numpy()  
        stats['n'][band] += 224*224
    nl_band = 'DMSP' if x['is_dmsp'] else 'VIIRS'
    stats['s1'][nl_band] += x['s1'][7].numpy()
    stats['s2'][nl_band] += x['s2'][7].numpy()
    stats['n'][nl_band] += 224*224
stats

Cause: could not parse the source code of <function <lambda> at 0x14ae37812c10>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x14ae37812c10>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 17min 53s, sys: 27.5 s, total: 18min 21s
Wall time: 4min 37s


{'s1': {'BLUE': 38476585.96664429,
  'GREEN': 57044804.188201904,
  'RED': 66842301.64126587,
  'SWIR1': 143199299.46417236,
  'SWIR2': 99276941.28448486,
  'TEMP1': 169744563908.5,
  'NIR': 151170596.80395508,
  'DMSP': 5702047176.0,
  'VIIRS': 738717936.027576},
 's2': {'BLUE': 3263325.6544721127,
  'GREEN': 7110897.5912714,
  'RED': 10824166.642980337,
  'SWIR1': 43282735.32255745,
  'SWIR2': 22762059.37821436,
  'TEMP1': 50823576248838.0,
  'NIR': 42658563.18126202,
  'DMSP': 235448106300.25,
  'VIIRS': 33761599188.670906},
 'n': {'BLUE': 569899008,
  'GREEN': 569899008,
  'RED': 569899008,
  'SWIR1': 569899008,
  'SWIR2': 569899008,
  'TEMP1': 569899008,
  'NIR': 569899008,
  'DMSP': 365481984,
  'VIIRS': 204417024}}

In [223]:
new_bandstats = {
    'A': {
        'means': {},
        'stds': {}
    },
    'B': {
        'means': {},
        'stds': {}
    },
    'C': {
        'means': {},
        'stds': {}
    },
    'D': {
        'means': {},
        'stds': {}
    },
    'E': {
        'means': {},
        'stds': {}
    }
}

# Calculate mean
for band in ['BLUE', 'GREEN', 'RED', 'SWIR1', 'SWIR2', 'TEMP1', 'NIR', 'DMSP', 'VIIRS']:
    new_bandstats['A']['means'][band] = np.float32(stats['s1'][band] / stats['n'][band])
    new_bandstats['A']['stds'][band] = np.float32(np.sqrt((stats['s2'][band] / stats['n'][band]) - np.square(stats['s1'][band] / stats['n'][band])))
    
print(new_bandstats['A'])
print(band_stats['A'])

{'means': {'BLUE': 0.06751475, 'GREEN': 0.10009634, 'RED': 0.11728798, 'SWIR1': 0.25127137, 'SWIR2': 0.17420094, 'TEMP1': 297.85025, 'NIR': 0.26525858, 'DMSP': 15.601445, 'VIIRS': 3.6137788}, 'stds': {'BLUE': 0.034174647, 'GREEN': 0.049580164, 'RED': 0.072364785, 'SWIR1': 0.11318473, 'SWIR2': 0.09795176, 'TEMP1': 21.568409, 'NIR': 0.067012966, 'DMSP': 20.020178, 'VIIRS': 12.332924}}
{'means': {'BLUE': 0.06614169065743208, 'GREEN': 0.09635988259340068, 'RED': 0.11381173286380034, 'SWIR1': 0.2403052018705142, 'SWIR2': 0.169467635861788, 'TEMP1': 281.2976763110107, 'NIR': 0.24095368620456603, 'DMSP': 13.069690484896771, 'VIIRS': 5.075756473790338}, 'stds': {'BLUE': 0.03512808462999218, 'GREEN': 0.050729950158437294, 'RED': 0.07239252056994558, 'SWIR1': 0.12444550248291769, 'SWIR2': 0.10850729467914068, 'TEMP1': 71.28094889057179, 'NIR': 0.09077465132496389, 'DMSP': 18.566259343422725, 'VIIRS': 15.861609457767129}}


In [34]:
!rm /mimer/NOBACKUP/groups/globalpoverty1/data/histograms/*/*.pickle

In [ ]:
10:07

In [ ]:
band_bin_edges = np.concatenate([
    [-1e5],
    np.arange(-5.0, 5.1, 0.1),
    [1e5]
])

def get_hist(img, label):
    bands = tf.reshape(img, (224 * 224, 8)).numpy()
    #print(np.min(bands, axis=0))
    img_hist = np.apply_along_axis(lambda band: np.histogram(band, bins=band_bin_edges)[0], 0, bands).T
    return img_hist, label.numpy()

for fold, fold_set in itertools.product(['A', 'B', 'C', 'D', 'E'], ['train', 'val', 'test']):
    hists = []
    labels = []
    fold_indices = content[fold][fold_set]
    fold_files = tfrecord_files[fold_indices]

    ds = data_handler.get_dataset(fold_files, batch_size=0, n_of_frames=1, 
                                  shuffle=False, normalize=True, 
                                  band_stats=band_stats[fold], max_epochs=1)
    for x in ds:
        img_hist, label = get_hist(*x)
        hists.append(img_hist)
        labels.append(label)

    hists = np.stack(hists)
    labels = np.stack(labels)

    res = (hists, labels)

    hist_file_name = os.path.join(data_dir, 'histograms', fold, fold_set + '.pickle')
    with open(hist_file_name, 'wb') as file:
        pickle.dump(res, file)
        
    print('Finished', hist_file_name)

Finished /mimer/NOBACKUP/groups/globalpoverty1/data/histograms/A/train.pickle


(45432, 816)

In [63]:
from sklearn.neighbors import KNeighborsRegressor

def knn_score(fold):
    hist_file_base = os.path.join(data_dir, 'histograms', fold, '{}.pickle')
    with open(hist_file_base.format('train'), 'rb') as handle:
        train_x, train_y = pickle.load(handle)

    with open(hist_file_base.format('val'), 'rb') as handle:
        val_x, val_y = pickle.load(handle)

    train_x = train_x.reshape(train_x.shape[0], -1)
    val_x = val_x.reshape(val_x.shape[0], -1)

    train_x = np.append(train_x, val_x, axis = 0)
    train_y = np.append(train_y, val_y, axis = 0)

    neigh = KNeighborsRegressor()
    neigh.fit(train_x, train_y)
    
    with open(hist_file_base.format('test'), 'rb') as handle:
        test_x, test_y = pickle.load(handle)

    test_x = test_x.reshape(test_x.shape[0], -1)
    return neigh.score(test_x, test_y)

for fold in ['A', 'B', 'C', 'D', 'E']:
    print(fold, knn_score(fold))

A 0.6199917708699771
B 0.7372431633298432
C 0.546750682391903
D 0.6296658187592816
E 0.717121424922049


In [57]:
with open(hist_file_base.format('test'), 'rb') as handle:
    test_x, test_y = pickle.load(handle)

test_x = test_x.reshape(test_x.shape[0], -1)
test_x.shape

(11357, 816)

In [58]:
neigh.score(test_x, test_y)

0.7106208818492195

In [54]:
neigh.score(test_x, test_y)

0.6457333620486714

In [237]:
np.append(fold_indices, content[fold]['val'])

array([    3,    14,    19, ..., 57174, 57181, 57191])

In [96]:
sum_hists = np.zeros((8, 102))
sum_hists[4] += hist
sum_hists[7] += hist / 0.7
sum_hists

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [ ]:
        label = x[1]
        bands = tf.reshape(img, (224 * 224, 8)).numpy()
        for i in range(8):
            band = bands[:, i]
            hist, _ = np.histogram(band, bins=band_bin_edges)
            print(hist)

In [5]:
ds = get_dataset(tfrecord_files, batch_size=0, n_of_frames=1, shuffle=False, normalize=False, max_epochs=1)

In [35]:
for x in ds.take(1):
    img = x[0]
    label = x[1]
    print(tf.reduce_mean(img, axis=(0, 1)))
    band_img = tf.reshape(img, (224 * 224, 8)).numpy()
    print(band_img.mean(axis=0))
    print(list(zip(band_img.min(axis=0), band_img.max(axis=0))))

tf.Tensor([-0.2249 -0.4148 -0.4316 -0.263  -0.3464  0.2396  3.344  -0.524 ], shape=(8,), dtype=float16)
[-0.2249 -0.4148 -0.4316 -0.263  -0.3464  0.2396  3.344  -0.524 ]
[(-0.8237, 1.132), (-0.979, 1.261), (-1.002, 1.799), (-1.174, 1.773), (-1.172, 2.146), (0.179, 0.2974), (2.0, 5.0), (-0.596, -0.4346)]


In [25]:
np.concatenate([
    [-1e5],
    np.arange(-5.0, 5.1, 0.1),
    [1e5]
])

array([-1.00000000e+05, -5.00000000e+00, -4.90000000e+00, -4.80000000e+00,
       -4.70000000e+00, -4.60000000e+00, -4.50000000e+00, -4.40000000e+00,
       -4.30000000e+00, -4.20000000e+00, -4.10000000e+00, -4.00000000e+00,
       -3.90000000e+00, -3.80000000e+00, -3.70000000e+00, -3.60000000e+00,
       -3.50000000e+00, -3.40000000e+00, -3.30000000e+00, -3.20000000e+00,
       -3.10000000e+00, -3.00000000e+00, -2.90000000e+00, -2.80000000e+00,
       -2.70000000e+00, -2.60000000e+00, -2.50000000e+00, -2.40000000e+00,
       -2.30000000e+00, -2.20000000e+00, -2.10000000e+00, -2.00000000e+00,
       -1.90000000e+00, -1.80000000e+00, -1.70000000e+00, -1.60000000e+00,
       -1.50000000e+00, -1.40000000e+00, -1.30000000e+00, -1.20000000e+00,
       -1.10000000e+00, -1.00000000e+00, -9.00000000e-01, -8.00000000e-01,
       -7.00000000e-01, -6.00000000e-01, -5.00000000e-01, -4.00000000e-01,
       -3.00000000e-01, -2.00000000e-01, -1.00000000e-01, -1.77635684e-14,
        1.00000000e-01,  

In [21]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sklearn.linear_model

from collections import defaultdict
from glob import glob
import re
import sys
import tensorflow as tf
from classification_models.tfkeras import Classifiers

In [ ]:
dhs_clusters = pd.read_csv('dhs_clusters.csv')

In [ ]:
dhs_ooc_folds = pd.read_pickle('dhs_ooc_folds.pkl')

In [ ]:
angola_indices = dhs_clusters[dhs_clusters['country']=='angola'].index
all_countries_indices = dhs_clusters[dhs_clusters['country']!='angola'].index

In [ ]:
all_countries = dhs_clusters['country'].unique()

In [ ]:
def get_country_indices(country):
  return dhs_clusters[dhs_clusters['country']==country].index

def get_all_country_indices():
  return  dhs_clusters.index

In [ ]:
get_country_indices('angola')

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            959, 960, 961, 962, 963, 964, 965, 966, 967, 968],
           dtype='int64', length=969)

In [ ]:
def train_linear_logo(features, labels, group_labels, cv_groups, test_groups,
                      weights=None, linear_model=sklearn.linear_model.Ridge,
                      plot=True, group_names=None, return_weights=False,
                      verbose=False):
    '''Leave-one-group-out cross-validated training of a linear model.
    Args
    - features: np.array, shape [N, D]
        each feature dim should be normalized to 0 mean, unit variance
    - labels: np.array, shape [N]
    - group_labels: np.array, shape [N], type np.int32
    - cv_groups: list of int, labels of groups to use for LOGO-CV
    - test_groups: list of int, labels of groups to test on
    - weights: np.array, shape [N]
    - linear_model: sklearn.linear_model
    - plot: bool, whether to plot MSE as a function of alpha
    - group_names: list of str, names of the groups, only used when plotting
    - return_weights: bool, whether to return the final trained model weights
    - verbose: bool
    Returns
    - test_preds: np.array, predictions on indices from test_groups
    - coefs: np.array, shape [D] (only returned if return_weights=True)
    - intercept: float (only returned if return_weights=True)
    '''
    cv_indices = np.isin(group_labels, cv_groups).nonzero()[0]
    test_indices = np.isin(group_labels, test_groups).nonzero()[0]

    X = cv_indices #features[cv_indices]
    y = labels[cv_indices]
    groups = group_labels[cv_indices]
    w = None if weights is None else weights[cv_indices]

    alphas = 2**np.arange(-5, 35, 3.0)
    preds = np.zeros([len(alphas), len(cv_indices)], dtype=np.float64)
    group_mses = np.zeros([len(alphas), len(cv_groups)], dtype=np.float64)
    leftout_group_labels = np.zeros(len(cv_groups), dtype=np.int32)
    logo = sklearn.model_selection.LeaveOneGroupOut()

    for i, alpha in enumerate(alphas):
        if verbose:
            print(f'\rAlpha: {alpha} ({i+1}/{len(alphas)})', end='')

        # set random_state for deterministic data shuffling
        model = linear_model(alpha=alpha, random_state=123)

        for g, (train_indices, val_indices) in enumerate(logo.split(X, groups=groups)):
            train_X, val_X = X[train_indices], X[val_indices]
            train_y, val_y = y[train_indices], y[val_indices]
            train_w = None if w is None else w[train_indices]
            val_w = None if w is None else w[val_indices]
            model.fit(X=train_X, y=train_y, sample_weight=train_w)
            val_preds = model.predict(val_X)
            preds[i, val_indices] = val_preds
            group_mses[i, g] = np.average((val_preds - val_y) ** 2, weights=val_w)
            leftout_group_labels[g] = groups[val_indices[0]]

    if verbose:
        print()
    mses = np.average((preds - y) ** 2, axis=1, weights=w)  # shape [num_alphas]

    if plot:
        h = max(3, len(group_names) * 0.2)
        fig, ax = plt.subplots(1, 1, figsize=[h*2, h], constrained_layout=True)
        for g in range(len(cv_groups)):
            group_name = group_names[leftout_group_labels[g]]
            ax.scatter(x=alphas, y=group_mses[:, g], label=group_name,
                       c=[cm.tab20.colors[g % 20]])
        ax.plot(alphas, mses, 'g-', label='Overall val mse')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Left-out Group')
        ax.set(xlabel='alpha', ylabel='mse')
        ax.set_xscale('log')
        ax.grid(True)
        plt.show()

    best_alpha = alphas[np.argmin(mses)]
    best_model = linear_model(alpha=best_alpha)
    best_model.fit(X=X, y=y, sample_weight=w)
    test_X, test_y, = features[test_indices], labels[test_indices]
    test_preds = best_model.predict(test_X)

    best_val_mse = np.min(mses)
    test_w = None if weights is None else weights[test_indices]
    test_mse = np.average((test_preds - test_y) ** 2, weights=test_w)
    print(f'best val mse: {best_val_mse:.3f}, best alpha: {best_alpha}, test mse: {test_mse:.3f}')

    if not return_weights:
        return test_preds
    else:
        coefs = best_model.coef_
        intercept = best_model.intercept_
        return test_preds, coefs, intercept

In [ ]:
def run_ridgecv_keep():
    '''
    For every country C (the test country):
      1. uses leave-one-country-out CV on all other countries
         to tune ridge model alpha parameter
      2. using best alpha, trains ridge model on all countries except C
      3. runs trained ridge model on C
    Saves predictions for each country on test.

    Args
    - model_name: str, format 'resnet_{bands}', e.g. 'resnet_ms'
    - labels: np.array, shape [num_examples]
    - locs: np.array, shape [num_examples, 2]
    - country_labels: np.array, shape [num_examples]
    - folds: dict, fold (str) => dict
    - keep_frac: float, fraction of non-test-country data to use for training
    - seed: int
    - savedir: str
    '''
    num_examples = len(all_countries)
    test_preds = np.zeros(num_examples, dtype=np.float32)
    FOLDS = ['A', 'B', 'C', 'D', 'E']
    all_countries_set = get_all_country_indices()

    countries_to_indices = defaultdict(list)
    for i, country in enumerate(all_countries):
        countries_to_indices[country].append(i)


    for f in FOLDS:
        for test_country in dhs_ooc_folds[f]['test']:
            print('test country:', test_country)

            test_country_set = {test_country}
            cv_countries_set = all_countries_set - test_country_set
            test_indices = get_indices_for_countries(test_country_set)
            test_preds[test_indices] = train_linear_logo(
                features=features[keep_subset_indices],
                labels=labels[keep_subset_indices],
                group_labels=country_labels[keep_subset_indices],
                cv_groups=countries_to_nums(cv_countries_set),
                test_groups=countries_to_nums(test_country_set),
                plot=False,
                group_names=COUNTRIES)

    # save preds on the test set
    os.makedirs(savedir, exist_ok=True)
    npz_path = os.path.join(savedir, 'test_preds_keep{k}_seed{s}.npz'.format(k=keep_frac, s=seed))
    print('Saving preds to:', savedir)
    np.savez_compressed(npz_path, test_preds=test_preds, labels=labels, locs=locs)
    '''
    for test_country in all_countries:
        print('test country:', test_country)

        test_country_set = {}
        cv_countries_set = all_countries_indices
        test_indices = angola_indices #get_indices_for_countries(test_country_set)
        ResNet18, _  = Classifiers.get('resnet18')
        img_net = ResNet18((224, 224, 3), weights='imagenet')
        features = img_net.layers[0].get_weights()
        test = train_linear_logo(
                    features= features,
                    labels=dhs_ooc_folds['A'],
                    group_labels=all_countries,
                    cv_groups=all_countries_indices,
                    test_groups=angola_indices,
                    plot=False,
                    group_names=all_countries)
        print(test)
    '''

In [ ]:
run_ridgecv_keep()

TypeError: ignored

In [ ]:
def load_npz(path, verbose=True, check=None):
    '''Loads .npz file into a dict.
    Args
    - path: str, path to .npz file
    - verbose: bool, whether to print out type and shape info
    - check: dict, key (str) => np.array, values to check
    Returns
    - result: dict
    '''
    result = {}
    with np.load(path) as npz:
        for key, value in npz.items():
            result[key] = value
            if verbose:
                print('{k}: dtype={d}, shape={s}'.format(k=key, d=value.dtype, s=value.shape))
    if check is not None:
        for key in check:
            assert key in result
            assert np.allclose(check[key], result[key])
    return result

In [ ]:
file_path = 'dhs_image_hists.npz'
npz = load_npz(file_path)

labels = npz['labels']
locs = npz['locs']
years = npz['years']

num_examples = len(labels)

image_hists: dtype=int64, shape=(19669, 8, 102)
labels: dtype=float32, shape=(19669,)
locs: dtype=float32, shape=(19669, 2)
years: dtype=int32, shape=(19669,)
nls_center: dtype=float32, shape=(19669,)
nls_mean: dtype=float32, shape=(19669,)


In [ ]:
npz

{'image_hists': array([[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        ...,
 
        [[ 0,  0,  0, ...,  2,  1, 27],
         [ 0,  0,  0, ...,  0,  5, 27],
         [ 0,  0,  0, ...,  0,  0, 10],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...